In [1]:
from composable import pipeable
from composable.strict import map
from toolz.curried import get
import re

In [2]:
with open('Sell_Short_Trades.txt') as f:
    content = f.read()
    f.close()
content

'                                                                                                                \n DOTC                                                                                 RUN DATE : 06/14/17 21:56 \n                                                                                             RPT DATE :06/14/17 \n                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    \n Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n                                                                                                                \n TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n                                                                    /17  /17  256945   

In [3]:
remove_spaces = content.replace(" ","")
remove_spaces

'\nDOTCRUNDATE:06/14/1721:56\nRPTDATE:06/14/17\nTTS0126:SELLSHORTTRADES&SHORTEXEMPT\nSymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr\n\nTradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr\nCERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS\n/17/1725694500\n0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO\nCERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786608704\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786609604\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX\n/17/1786978023\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX\n/17/1789288829\n0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\n854300\nCERSSSNEW1002.3600002.35

In [4]:
culled_split=remove_spaces.split("\n")
culled_split

['',
 'DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 '',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnit

In [5]:
empty_spots = lambda l: [len(line.strip())==0 for line in l]


In [6]:
empty_spots(culled_split[:10])

[True, False, False, False, False, True, False, False, False, False]

In [7]:
culled_lines = [line for line in culled_split if empty_spots(line)]
culled_lines

['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.0100

In [ ]:
example_block = culled_lines
example_block[:10]

In [8]:
pagebreak_example = culled_lines[45:58]
pagebreak_example

['854300',
 '1of77',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW1002.3600002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX',
 '/17/1790916802',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX',
 '/17/1790917702',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3650002.3500002.36000006/1406/19170614990000310:24:ContraSSFREX']

In [10]:
[line for line in culled_lines if line.strip().endswith('of77')][:20]

['1of77',
 '2of77',
 '3of77',
 '4of77',
 '5of77',
 '6of77',
 '7of77',
 '8of77',
 '9of77',
 '10of77',
 '11of77',
 '12of77',
 '13of77',
 '14of77',
 '15of77',
 '16of77',
 '17of77',
 '18of77',
 '19of77',
 '20of77']

In [11]:
remove_pagebreaks = [line for line in culled_lines if not line.strip().endswith('of77')]
remove_pagebreaks

['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW2,7562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.0100

In [12]:
remove_commas = [line.replace(",","") for line in remove_pagebreaks]
remove_commas



['DOTCRUNDATE:06/14/1721:56',
 'RPTDATE:06/14/17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006/1406/19170614990000309:30:CustSS',
 '/17/1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:20:ContraSSFREX',
 '/17/1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006/1406/19170614990000310:22:ContraSSFREX',
 '/17/1789288829',
 '0704PerUnitTERM0.01000

In [13]:
remove_backslash = [line.replace("/"," ") for line in remove_commas]
remove_backslash

['DOTCRUNDATE:06 14 1721:56',
 'RPTDATE:06 14 17',
 'TTS0126:SELLSHORTTRADES&SHORTEXEMPT',
 'SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.01000

In [14]:
data_lines = remove_backslash[3:]
data_lines

['SymbolSideCxlQtyPriceBidAskT-DatS-DatTradeIDTradeTiSSExbkr',
 'TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 191

In [15]:
working_data = data_lines

In [16]:
first_columnrow = lambda l: [line.strip().startswith('Symbol') for line in l]

In [17]:
first_columnrow(working_data)

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [18]:
first_columnrow_remove = [line for line in working_data if not line.strip().startswith('Symbol')]
first_columnrow_remove


['TradeCommTypeSourceCommissionAccountOrderIDGTLTrailerInfoClr',
 'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUn

In [19]:
second_columnrow = lambda l: [line.strip().startswith('TradeComm') for line in l]
second_columnrow(first_columnrow_remove)

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [20]:
second_columnrow_remove = [line for line in first_columnrow_remove if not line.strip().startswith('TradeComm')]
second_columnrow_remove

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000

In [21]:
data = second_columnrow_remove
data

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000

In [22]:
numbers_newline = re.compile(r' 17 17.*')

In [23]:
test = [line for line in second_columnrow_remove if numbers_newline.search(line) and line.strip().startswith('0704')]
test

['0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1790915402',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1790922402',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1701100807',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1717918809',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1726421956',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1726438257',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1727181934',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1746951220',
 '0704PerUnitTERM0.030000EEFFGG1706149900004#STA=MSCO 17 1748011528',
 '0704PerUnitTERM0.030000EEFFGG1706149900004#STA=MSCO 17 1748041330',
 '0704PerUnitTERM0.030000EEFFGG1706149900004#STA=MSCO 17 1748190739',
 '0704PerUnitTERM0.030000EEFFGG1706149900004#STA=MSCO 17 1748310249',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1750224124',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS 17 1750295929',
 '0704PerUnitTERM0.030000EEFFG

In [ ]:
test3 = [line.replace('CERS','CERS\n') for line in test]
tuple(test3)

In [ ]:
test4 = ",".join(test3)
test4

In [ ]:
test5=test4.split("\n")
test4

In [24]:
stop_cond = lambda w: len(w)==2

In [25]:
get_layer = lambda d: d[:4] if len(d[3])==6 else d[:3]

In [26]:
get_remaining = lambda r: r[len(get_layer(r)):]

In [27]:
remaining_lines = data

In [28]:
remaining_lines[-5:]

[' 17 1784408859',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'TOTALTRADES:913',
 '********ENDOFREPORT*********']

In [29]:
get_layer(remaining_lines)

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS',
 ' 17 1725694500',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO']

In [30]:
remaining_lines = get_remaining(remaining_lines)
remaining_lines

['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786609604',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786978023',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
 ' 17 1789288829',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790911901',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
 ' 17 1790912801',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300',
 

In [31]:
get_layer(remaining_lines)

['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
 ' 17 1786608704',
 '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 '854300']

In [33]:
new_seq = []
while not stop_cond(remaining_lines):
    new_seq = new_seq + [get_layer(remaining_lines)]
    remaining_lines = get_remaining(remaining_lines)


In [34]:
l = new_seq
l

[['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786608704',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786609604',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX',
  ' 17 1786978023',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX',
  ' 17 1789288829',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
  '854300'],
 ['CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
  ' 17 1790911901',
  '0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS'],
 ['CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX',
  ' 17 1790912801',
  '0704PerUnitTERM0.010000AABBC1706149900

In [35]:
joined_data = [",".join(line) for line in l]
joined_data

['CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790912801,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:Co

In [36]:
text_content = "\n".join(joined_data)
text_content

'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\nCERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790912801,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790

In [149]:
with open('Project_pt2.txt',"w") as outfile:
    outfile.write(text_content)
    

In [ ]:
#numbers_notneeded =  lambda x: [line.strip().startswith("854300") or line.strip().startswith("473640")  for line in x]
#numbers_notneeded(second_columnrow_remove)

In [ ]:
#numbers_cleaning = [line for line in working_data if not line.strip().startswith("854300")]
#numbers_notneeded = [line for line in numbers_cleaning if not line.strip().startswith("473640")]
#numbers_notneeded


In [74]:
with open('Project_pt2.txt') as f:
    data_parse = f.read()
    f.close()
data_parse

'CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO\nCERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300\nCERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS\nCERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790912801,0704P

In [85]:
data_parse = data_parse.split('\n')
data_parse

['CERSSSNEW27562.4000002.3400002.45000006 1406 19170614990000309:30:CustSS, 17 1725694500,0704PerUnitTERM0.010000AABBC1706149900003#STA=GSCO',
 'CERSSSNEW1002.36000018422.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786608704,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW2002.3500002.3600002.37000006 1406 19170614990000310:20:ContraSSFREX, 17 1786609604,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:20:ContraSSFREX, 17 1786978023,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3500002.3500002.36000006 1406 19170614990000310:22:ContraSSFREX, 17 1789288829,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS,854300',
 'CERSSSNEW1002.3600002.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 17 1790911901,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS',
 'CERSSSNEW1002.36000085432.3500002.36000006 1406 19170614990000310:24:ContraSSFREX, 

In [86]:
get_symbol = re.compile(r'CERS|S')
trade_symbols = [get_symbol.search(line).group() for line in data_parse]
trade_symbols

['CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'CERS',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 'S',
 

In [89]:
len(data_parse)

913

In [88]:
len(trade_symbols)

913

In [68]:
get_side = re.compile(r'SS')
trade_side = [get_side.search(line).group() for line in joined_data]
trade_side

['SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',
 'SS',

In [69]:
len(trade_side)

912

In [108]:
get_side = re.compile(r'2756|1100|1000|2700|99|2800|1487|2013|34|66|90|40|48|46|\d{1,3}')
trade_quantity = [get_side.search(line).group() for line in data_parse]
trade_quantity

['2756',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '200',
 '100',
 '300',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '200',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '300',
 '300',
 '100',
 '100',
 '100',
 '200',
 '100',
 '200',
 '100',
 '12',
 '199',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '200',
 '212',
 '792',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '202',
 '982',
 '742',
 '262',
 '100',
 '100',
 '100',
 '100',
 '100',
 '700',
 '100',
 '100',
 '100',
 '100',
 '100',
 '40',
 '100',
 '100',
 '300',
 '200',
 '100',
 '300',
 '100',
 '200',
 '100',
 '100',
 '100',
 '200',
 '100',
 '100',
 '100',
 '100',
 '300',
 '100',
 '100',
 '200',
 '100',
 '100',
 '100',
 '200',
 

In [111]:
get_price = re.compile(r'\d\.\d{1,3}')
trade_price = [get_price.search(line).group() for line in data_parse]
trade_price

['2.400',
 '2.360',
 '2.350',
 '2.350',
 '2.350',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.365',
 '2.365',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.365',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.360',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '2.350',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',
 '8.210',


In [113]:
get_bid= re.compile(r'0\d\.\d\d')
trade_bid = [get_bid.search(line).group() for line in data_parse]
trade_bid

['02.34',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '02.36',
 '02.35',
 '02.35',
 '02.35',
 '02.35',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',


In [116]:
trade_bid_fixing= [line.replace('02','2') for line in trade_bid]
trade_bid_fixing

['2.34',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 '08.21',
 

In [117]:
trade_bid_cleaned = [line.replace('08',"8") for line in trade_bid_fixing]
trade_bid_cleaned

['2.34',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '2.36',
 '2.35',
 '2.35',
 '2.35',
 '2.35',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 '8.21',
 

GET ASKING PRICE STILL


In [118]:
get_TdatsTrade = re.compile(r'1406')
trade_tDats = [get_TdatsTrade.search(line).group() for line in data_parse]
trade_tDats

['1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 '1406',
 

In [119]:
len(trade_tDats)

913

In [129]:
get_tradeTis = re.compile(r'191\d{1,12}')
trade_tradeTis = [get_tradeTis.search(line).group() for line in data_parse]
trade_tradeTis

['191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900003',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',
 '191706149900004',


In [171]:
get_exBkr = re.compile(r'\d{1,2}:.+? ')
trade_exBkr= [get_exBkr.search(line).group() for line in data_parse]
trade_exBkr

['09:30:CustSS, ',
 '10:20:ContraSSFREX, ',
 '10:20:ContraSSFREX, ',
 '10:20:ContraSSFREX, ',
 '10:22:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS ',
 '10:24:ContraSSFREX, ',
 '10:24:ContraSSFREX, ',
 '10:25:ContraSSFREX, ',
 '10:27:ContraSSFREX, ',
 '10:27:ContraSSFREX, ',
 '10:27:ContraSSFREX, ',
 '10:27:ContraSSFREX, ',
 '10:27:ContraSSFREX, ',
 '10:29:ContraSSFREX, ',
 '10:30:ContraSSFREX, ',
 '10:33:ContraSSFREX, ',
 '10:33:ContraSSFREX,0704PerUnitTERM0.010000AABBC1706149900003#STA=CERS ',
 '10:33:ContraSSFREX, ',
 '10:33:ContraSSFREX, ',
 '10:36:ContraSSFREX, ',
 '10:3

FIX THIS ABOVE

In [174]:
get_tradeComm = re.compile(r'0704')
trade_CommType= [get_tradeComm.search(line).group() for line in data_parse]
trade_CommType

['0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 '0704',
 

In [175]:
len(trade_CommType)

913

In [177]:
get_sourceComm = re.compile(r'PerUnit')
trade_sourceComm= [get_sourceComm.search(line).group() for line in data_parse]
trade_sourceComm

['PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'PerUnit',
 'Pe

In [178]:
len(trade_sourceComm)

913

In [181]:
get_account = re.compile(r'TERM')
trade_account= [get_account.search(line).group() for line in data_parse]
trade_account

['TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 'TERM',
 

In [182]:
len(trade_account)

913

In [186]:
get_orderID = re.compile(r'TERM\d.\d{1,5}')
trade_orderID= [get_orderID.search(line).group() for line in data_parse]
trade_orderID

['TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.01000',
 'TERM0.

In [187]:
trade_orderID_clean = [line.replace('TERM',"") for line in trade_orderID]
trade_orderID_clean

['0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.01000',
 '0.

In [189]:
get_gtl = re.compile(r'AABBC|EEFFGG')
trade_gtl= [get_gtl.search(line).group() for line in data_parse]
trade_gtl

['AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'AABBC',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 'EEFFGG',
 '

In [190]:
len(trade_gtl)

913

In [207]:
get_trailerInfo= re.compile(r'[A-Z]17.+')
trade_trailerInfo= [get_trailerInfo.search(line).group() for line in data_parse]
trade_trailerInfo

['C1706149900003#STA=GSCO',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS 17 1790915402,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS 17 1790922402,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA=CERS,854300',
 'C1706149900003#STA

Fix this above
